In [3]:
!pip3 install beautifulsoup4
!pip3 install requests
import sys
from bs4 import BeautifulSoup
import functools
import requests

You should consider upgrading via the 'c:\users\zya\globalpyve\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\zya\globalpyve\scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
def loadElements(inputFile):
    html = open(inputFile, encoding='utf8').read()
    bs = BeautifulSoup(html)
    elements = bs.select('#anchor_sound_list > div.sound-list._is > ul > li > div.text.lF_')
    return elements

In [8]:
elements = []
for i in range(1, 7+1):
    inputFile = 'page{}.html'.format(i)
    es = loadElements(inputFile)
    elements.extend(es)

In [9]:
def parseElement(x):
    y = next(x.children)
    link = y.attrs['href']
    title = y.text
    item_id = link.split('/')[-1]
    return {'item_id': item_id,'title':title}

In [10]:
items = [parseElement(x) for x in elements]

In [32]:
# https://blog.csdn.net/BigBoy_Coder/article/details/103406332
import requests
import time
import hashlib
import random
 
 
class ximalaya(object):
 
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36"
        }
 
    def getServerTime(self):
        """
        获取喜马拉雅服务器的时间戳
        :return:
        """
        # 这个地址就是返回服务器时间戳的接口
        serverTimeUrl = "https://www.ximalaya.com/revision/time"
        response = requests.get(serverTimeUrl,headers = self.headers)
        return response.text
 
    def getSign(self,serverTime):
        """
        生成 xm-sign
        规则是 md5(ximalaya-服务器时间戳)(100以内随机数)服务器时间戳(100以内随机数)现在时间戳
        :param serverTime:
        :return:
        """
        nowTime = str(round(time.time()*1000))
 
        sign = str(hashlib.md5("himalaya-{}".format(serverTime).encode()).hexdigest()) + "({})".format(str(round(random.random()*100))) + serverTime + "({})".format(str(round(random.random()*100))) + nowTime
        # 将xm-sign添加到请求头中
        self.headers["xm-sign"] = sign
        return sign

In [29]:
x = ximalaya()
x.getSign(x.getServerTime())

'8027f59161034deedd03f21bd6bbc653(77)1611118563632(21)1611118563899'

In [30]:
@functools.lru_cache()
def getDownloadLink(item_id):
    url = 'https://www.ximalaya.com/revision/play/v1/audio?id={}&ptype=1'.format(item_id)
    print(url)
    x = ximalaya()
    sign = x.getSign(x.getServerTime())
    r = requests.get(url, headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36', 'Content-Type': 'application/json',
        'xm-sign':sign,
        'cookie':'_xmLog=h5&ba306987-7f3f-4929-917d-e761b482168d&2.2.12; Hm_lvt_4a7d8ec50cfd6af753c4f8aee3425070=1611117155; wb_nickname_1=dirtysalt; wb_avatar_1=https%3A%2F%2Ftvax4.sinaimg.cn%2Fcrop.0.0.512.512.180%2F7a1ecb3ely8g4j9uk9ijcj20e80e8t99.jpg%3FKID%3Dimgbed%2Ctva%26Expires%3D1611127971%26ssig%3Dy%252BaH962tnH; trackType=web; fds_otp=6240493213403637617; x_xmly_traffic=utm_source%253A%2526utm_medium%253A%2526utm_campaign%253A%2526utm_content%253A%2526utm_term%253A%2526utm_from%253A; 1&remember_me=y; 1&_token=55621669&1E9FD900140NC550B690C0DDAE717E13971709F994E274E1653C59C8A0FAC9B05C1FD725104C39M24020C8EC725C6B_; 1_l_flag=55621669&1E9FD900140NC550B690C0DDAE717E13971709F994E274E1653C59C8A0FAC9B05C1FD725104C39M24020C8EC725C6B__2021-01-2012:35:41; Hm_lpvt_4a7d8ec50cfd6af753c4f8aee3425070=1611117347',
        'referer':'https://www.ximalaya.com/ertong/29122679/360821293',
    })
    print(r.status_code, r.content)
    link = r.json()['data']['src']    
    return link

In [31]:
getDownloadLink('360821293')

https://www.ximalaya.com/revision/play/v1/audio?id=360821293&ptype=1
200 b'{"ret":200,"data":{"trackId":360821293,"canPlay":true,"isPaid":false,"hasBuy":true,"src":"https://aod.cos.tx.xmcdn.com/storages/a177-audiofreehighqps/FC/A1/CMCoOSQDloqnABR3nwBrXce5.m4a","albumIsSample":false,"sampleDuration":49,"isBaiduMusic":false,"firstPlayStatus":true,"isVipFree":false,"isXimiAhead":false,"isAlbumTimeLimited":false,"ximiVipFreeType":0}}'


'https://aod.cos.tx.xmcdn.com/storages/a177-audiofreehighqps/FC/A1/CMCoOSQDloqnABR3nwBrXce5.m4a'

In [33]:
len(items)

196

In [34]:
len(set([x['title'] for x in items]))

196

In [ ]:
for it in items:
    if 'link' not in it:
        link = getDownloadLink(it['item_id'])
#         print('OK!!!')
        it['link'] = link
    else:
        print(it)

In [ ]:
with open('download.sh', 'w') as fh:
    for it in items:
        s = """
        if [ ! -f "audio/{title}.m4a" ]; then
            wget "{url}" -O temp.m4a
            mv temp.m4a "audio/{title}.m4a"
        fi
        """.format(title = it['title'], url = it['link'])
        fh.write(s)